In [ ]:
import logging

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()

In [ ]:
from azureml.core import Datastore

sql_datastore = Datastore.get(workspace=ws, datastore_name="ado_sql_datastore")

In [ ]:
from azureml.core import Dataset
from azureml.data.datapath import DataPath
query = DataPath(sql_datastore, 'SELECT *  FROM Improvements')
improvements_sql_ds = Dataset.Tabular.from_sql_query(query)

improvements_sql_ds.register(workspace=ws,
                          name="ai_ag_ado_improvements",
                          description = "Improvements from Azure DevOps")

In [ ]:
from azureml.core import Dataset

query_string = 'SELECT *, (POWER(1.5,MitigationScore) * POWER(2,Priority) * POWER(6.585, IsBlocker)) as dc_impact_score FROM FeedbackItems'

query = DataPath(sql_datastore, query_string)
feedback_sql_ds = Dataset.Tabular.from_sql_query(query)

feedback_sql_ds.register(workspace=ws,
                         name="ai_ag_ado_feedack",
                         description = "Feedback from Azure DevOps")

In [ ]:
# feedback_sql_pd = feedback_sql_ds.to_pandas_dataframe()
# 
# label ="dc_impact_score"
# 
# def dc_impact_score_calculation(mitigation_score, priority, is_blocker):
#     return (1.5**mitigation_score) * (2**priority) * (6.585**is_blocker)
#     
# 
# feedback_sql_pd[label] = dc_impact_score_calculation(feedback_sql_pd['MitigationScore'], feedback_sql_pd['Priority'], feedback_sql_pd['IsBlocker'])
# 
# file="temp"
# feedback_sql_pd.to_csv(file)
# feedback_sql_ds_labeled = Dataset.Tabular.from_delimited_files(path=file)

In [ ]:
def split_dataset(dataset):
    # Split the dataset into train and test datasets
    train_data, test_data = dataset.random_split(percentage=0.8, seed=223)

    # Register the train dataset with your workspace
    train_data.register(workspace = ws, 
                        name = 'ai_ag_ado_feedack_train_dataset',
                        description = 'Feedback from Azure DevOps training data',
                        create_new_version=True)

    # Register the test dataset with your workspace
    test_data.register(workspace = ws, 
                       name = 'ai_ag_ado_feedack_test_dataset', 
                       description = 'Feedback from Azure DevOps test data')
    return train_data, test_data
    
train_data, test_data = split_dataset(feedback_sql_ds)    

In [ ]:

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

# Choose a name for your cluster.
amlcompute_cluster_name = "cpu-cluster"

found = False
# Check if this compute target already exists in the workspace.
cts = ws.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
    found = True
    print('Found existing compute target.')
    compute_target = cts[amlcompute_cluster_name]

if not found:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", # for GPU, use "STANDARD_NC6"
                                                                #vm_priority = 'lowpriority', # optional
                                                                max_nodes = 4)

    # Create the cluster.\n",
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)

print('Checking cluster status...')
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
compute_target.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)

# For a more detailed view of current AmlCompute status, use get_status().

In [ ]:
import logging
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task = 'regression',
                             debug_log = 'automl_errors.log',
                             compute_target=compute_target,
                             training_data = train_data,
                             label_column_name = label,
                             "verbosity": logging.INFO,
                             "enable_early_stopping": True, 
                             "experiment_timeout_minutes" : 10,
                             "max_concurrent_iterations": 4,
                             "max_cores_per_iteration": -1,
                             "n_cross_validations": 5,
                             "primary_metric": 'normalized_root_mean_squared_error'
                            )

In [ ]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "ai-impact-score-experiment-dc-sql")

runs = experiment.get_runs()

if not runs:
    remote_run = experiment.submit(automl_config, show_output=True)
else:
    for run in runs:
        remote_run = run
        break;

In [ ]:
children = list(remote_run.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}
    metricslist[int(properties['iteration'])] = metrics

rundata = pd.DataFrame(metricslist).sort_index(1)
rundata

In [ ]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

In [ ]:
y_pred_train = fitted_model.predict(x_train)
y_residual_train = y_train - y_pred_train

y_pred_test = fitted_model.predict(x_test)
y_residual_test = y_test - y_pred_test

In [ ]:
%matplotlib inline
from sklearn.metrics import mean_squared_error, r2_score

# Set up a multi-plot chart.
f, (a0, a1) = plt.subplots(1, 2, gridspec_kw = {'width_ratios':[1, 1], 'wspace':0, 'hspace': 0})
f.suptitle('Regression Residual Values', fontsize = 18)
f.set_figheight(6)
f.set_figwidth(16)

# Plot residual values of training set.
a0.axis([0, 360, -200, 200])
a0.plot(y_residual_train, 'bo', alpha = 0.5)
a0.plot([-10,360],[0,0], 'r-', lw = 3)
a0.text(16,170,'RMSE = {0:.2f}'.format(np.sqrt(mean_squared_error(y_train, y_pred_train))), fontsize = 12)
a0.text(16,140,'R2 score = {0:.2f}'.format(r2_score(y_train, y_pred_train)), fontsize = 12)
a0.set_xlabel('Training samples', fontsize = 12)
a0.set_ylabel('Residual Values', fontsize = 12)

# Plot a histogram.
a0.hist(y_residual_train, orientation = 'horizontal', color = 'b', bins = 10, histtype = 'step')
a0.hist(y_residual_train, orientation = 'horizontal', color = 'b', alpha = 0.2, bins = 10)

# Plot residual values of test set.
a1.axis([0, 90, -200, 200])
a1.plot(y_residual_test, 'bo', alpha = 0.5)
a1.plot([-10,360],[0,0], 'r-', lw = 3)
a1.text(5,170,'RMSE = {0:.2f}'.format(np.sqrt(mean_squared_error(y_test, y_pred_test))), fontsize = 12)
a1.text(5,140,'R2 score = {0:.2f}'.format(r2_score(y_test, y_pred_test)), fontsize = 12)
a1.set_xlabel('Test samples', fontsize = 12)
a1.set_yticklabels([])

# Plot a histogram.
a1.hist(y_residual_test, orientation = 'horizontal', color = 'b', bins = 10, histtype = 'step')
a1.hist(y_residual_test, orientation = 'horizontal', color = 'b', alpha = 0.2, bins = 10)

plt.show()

In [ ]:
run.get_file_names()

In [ ]:
model = run.register_model(model_name='best_sql_dc_impact_score_model', model_path='./outputs/model.pkl')

In [ ]:
print("Registered model:\n --> Name: {}\n --> Version: {}\n --> URL: {}".format(model.name, model.version, model.url))